# 股價爬蟲

In [17]:
import time
import requests
import pandas as pd
import datetime
import re
import numpy as np

In [18]:
df = []
for date in pd.date_range('2021-06-01', '2022-04-01', freq='MS'):
    url = 'https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date={}'.format(date.strftime('%Y%m%d'))
    print(url)
    rs = requests.Session()
    resp = rs.get(url)
    ndf = pd.DataFrame(resp.json()['data'], 
             columns=resp.json()['fields'])
    ndf['title'] = resp.json()['title']
    time.sleep(4)
    df.append(ndf)
df = pd.concat(df, ignore_index=True)
df

https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20210601
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20210701
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20210801
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20210901
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20211001
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20211101
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20211201
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20220101
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20220201
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20220301
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20220401


,Date,Opening Index,Highest Index,Lowest Index,Closing Index,title
0,2021/06/01,"17,098.49","17,184.30","17,060.45","17,162.38",2021/06 TAIEX Total Index Historical Data
1,2021/06/02,"17,192.91","17,274.66","17,056.42","17,165.04",2021/06 TAIEX Total Index Historical Data
2,2021/06/03,"17,201.30","17,311.28","17,198.89","17,246.16",2021/06 TAIEX Total Index Historical Data
3,2021/06/04,"17,225.06","17,225.06","17,084.49","17,147.41",2021/06 TAIEX Total Index Historical Data
4,2021/06/07,"17,176.09","17,189.23","16,775.85","17,083.91",2021/06 TAIEX Total Index Historical Data
...,...,...,...,...,...,...
208,2022/04/08,"17,210.40","17,316.69","17,210.40","17,284.54",2022/04 TAIEX Total Index Historical Data
209,2022/04/11,"17,273.05","17,316.35","17,046.67","17,048.37",2022/04 TAIEX Total Index Historical Data
210,2022/04/12,"17,005.84","17,100.85","16,905.13","16,990.91",2022/04 TAIEX Total Index Historical Data
211,2022/04/13,"17,080.40","17,333.21","17,080.40","17,301.65",2022/04 TAIEX Total Index Historical Data


In [19]:
df['Date'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d'))
df['Opening Index'] = df['Opening Index'].apply(lambda x: float(re.sub(',', '', x)))
df['Highest Index'] = df['Highest Index'].apply(lambda x: float(re.sub(',', '', x)))
df['Lowest Index'] = df['Lowest Index'].apply(lambda x: float(re.sub(',', '', x)))
df['Closing Index'] = df['Closing Index'].apply(lambda x: float(re.sub(',', '', x)))
df.columns = [col.upper() for col in df.columns]
df.columns = [re.sub(' ', '_', col) for col in df.columns]
df

,DATE,OPENING_INDEX,HIGHEST_INDEX,LOWEST_INDEX,CLOSING_INDEX,TITLE
0,2021-06-01,17098.49,17184.30,17060.45,17162.38,2021/06 TAIEX Total Index Historical Data
1,2021-06-02,17192.91,17274.66,17056.42,17165.04,2021/06 TAIEX Total Index Historical Data
2,2021-06-03,17201.30,17311.28,17198.89,17246.16,2021/06 TAIEX Total Index Historical Data
3,2021-06-04,17225.06,17225.06,17084.49,17147.41,2021/06 TAIEX Total Index Historical Data
4,2021-06-07,17176.09,17189.23,16775.85,17083.91,2021/06 TAIEX Total Index Historical Data
...,...,...,...,...,...,...
208,2022-04-08,17210.40,17316.69,17210.40,17284.54,2022/04 TAIEX Total Index Historical Data
209,2022-04-11,17273.05,17316.35,17046.67,17048.37,2022/04 TAIEX Total Index Historical Data
210,2022-04-12,17005.84,17100.85,16905.13,16990.91,2022/04 TAIEX Total Index Historical Data
211,2022-04-13,17080.40,17333.21,17080.40,17301.65,2022/04 TAIEX Total Index Historical Data


In [20]:
# EDA 繪製股價變化圖

## 活動
- 如何計算連續的休假天數?

In [21]:
stock_price = pd.DataFrame(pd.date_range('2019-12-01', '2022-04-01', freq='D'), columns=['DATE'])
stock_price = pd.merge(stock_price, df[['DATE', 'CLOSING_INDEX']], how='left', on='DATE')
stock_price = stock_price.loc[stock_price['CLOSING_INDEX'].notnull()]
stock_price['LAST_DATE'] = stock_price['DATE'].shift(1)
stock_price['DAYS'] = stock_price['DATE'] - stock_price['LAST_DATE']
stock_price['DAYS'] = stock_price['DAYS'].apply(lambda x: x.days)
stock_price['DAYS'] = stock_price['DAYS']-1
stock_price.head(30)

,DATE,CLOSING_INDEX,LAST_DATE,DAYS
548,2021-06-01,17162.38,NaT,NaN
549,2021-06-02,17165.04,2021-06-01,0.0
550,2021-06-03,17246.16,2021-06-02,0.0
551,2021-06-04,17147.41,2021-06-03,0.0
554,2021-06-07,17083.91,2021-06-04,2.0
555,2021-06-08,17076.21,2021-06-07,0.0
556,2021-06-09,16966.22,2021-06-08,0.0
557,2021-06-10,17159.22,2021-06-09,0.0
558,2021-06-11,17213.52,2021-06-10,0.0
562,2021-06-15,17371.29,2021-06-11,3.0


In [22]:
stock_price['DAYS'].value_counts()

0.0     163
2.0      36
3.0       4
4.0       1
11.0      1
Name: DAYS, dtype: int64

# convex function

In [23]:
closings = [2, np.nan, 6, 8, 10, np.nan, np.nan, np.nan, 18, 20]
for i in range(len(closings)):
    if str(closings[i]) != 'nan':
        pass
    if str(closings[i]) == 'nan':
        last_ava = closings[i-1]
        j = 1
        while True:
            if str(closings[i+j]) != 'nan':
                next_ava = closings[i+j]
                break
            j = j+1
        closings[i] = (last_ava+next_ava)/2
closings

[2, 4.0, 6, 8, 10, 14.0, 16.0, 17.0, 18, 20]

In [28]:
## 如何處理遺漏值 - 週末、假日...
stock_price = pd.DataFrame(pd.date_range('2021-06-01', '2022-04-01', freq='D'), columns=['DATE'])
stock_price = pd.merge(stock_price, df[['DATE', 'CLOSING_INDEX']], how='left', on='DATE')
stock_price.head(50)

,DATE,CLOSING_INDEX
0,2021-06-01,17162.38
1,2021-06-02,17165.04
2,2021-06-03,17246.16
3,2021-06-04,17147.41
4,2021-06-05,NaN
5,2021-06-06,NaN
6,2021-06-07,17083.91
7,2021-06-08,17076.21
8,2021-06-09,16966.22
9,2021-06-10,17159.22


In [29]:
stock_price = stock_price.loc[stock_price['DATE']!='2019-12-01']
for i in range(stock_price.shape[0]):
    if str(stock_price['CLOSING_INDEX'].values[i]) != 'nan':
        pass
    if str(stock_price['CLOSING_INDEX'].values[i]) == 'nan':
        last_ava = stock_price['CLOSING_INDEX'].values[i-1]
        
        j = 1
        while True:
            if str(stock_price['CLOSING_INDEX'].values[i+j]) != 'nan':
                next_ava = stock_price['CLOSING_INDEX'].values[i+j]
                break
            j = j+1
        stock_price.iloc[i, 1] = (last_ava+next_ava)/2
stock_price

,DATE,CLOSING_INDEX
0,2021-06-01,17162.38
1,2021-06-02,17165.04
2,2021-06-03,17246.16
3,2021-06-04,17147.41
4,2021-06-05,17115.66
...,...,...
300,2022-03-28,17520.01
301,2022-03-29,17548.66
302,2022-03-30,17740.56
303,2022-03-31,17693.47


In [30]:
stock_price.head(20)

,DATE,CLOSING_INDEX
0,2021-06-01,17162.38000
1,2021-06-02,17165.04000
2,2021-06-03,17246.16000
3,2021-06-04,17147.41000
4,2021-06-05,17115.66000
5,2021-06-06,17099.78500
6,2021-06-07,17083.91000
7,2021-06-08,17076.21000
8,2021-06-09,16966.22000
9,2021-06-10,17159.22000


In [31]:
# 驗證
5	2019-12-06	11609.64
6	2019-12-07	NaN
7	2019-12-08	NaN
8	2019-12-09	11660.77

SyntaxError: invalid syntax (2307642925.py, line 2)

In [32]:
(11609.64 + 11660.77)/2

11635.205

In [33]:
(11635.205 + 11660.77)/2

11647.9875

In [34]:
stock_price.to_pickle('./stock_price_by_date.pickle')